# Data Preparation Toward Analysing  Crime and Tweet sentiments 

Check the Medium article for more details.


 ## Police Data Preparation
 

In [60]:
import pandas as pd
print(pd.__version__)
 

1.0.3


In [61]:
police_data = pd.read_csv('../data/police_data.csv')
 

In [62]:
police_data.head()

,Table 1,ANZSOC Division,Year Month,Month Year,ANZSOC Group,ANZSOC Subdivision,Area Unit,Location Type,Locn Type Division,Meshblock,Occurrence Day Of Week,Occurrence Hour Of Day,Territorial Authority,Weapon,Year Month (copy 2),Number of Records,Victimisations
0,1 Victimisation Time and Place,Theft and Related Offences,July 2014,July 2014,Theft From Retail Premises,Theft (Except Motor Vehicles),Bethlehem.,.,Other Location,1190602,Wednesday,22.0,Tauranga City.,Not Applicable,July 2014,1,1
1,1 Victimisation Time and Place,Theft and Related Offences,July 2014,July 2014,Theft From Retail Premises,Theft (Except Motor Vehicles),Methven.,.,Other Location,2752700,Wednesday,99.0,Ashburton District.,Not Applicable,July 2014,1,1
2,1 Victimisation Time and Place,Theft and Related Offences,July 2014,July 2014,Theft From Retail Premises,Theft (Except Motor Vehicles),Wesley.,.,Other Location,579200,Wednesday,99.0,Auckland.,Not Applicable,July 2014,1,1
3,1 Victimisation Time and Place,Theft and Related Offences,July 2014,July 2014,Theft From Retail Premises,Theft (Except Motor Vehicles),Riccarton.,.,Other Location,2472800,Wednesday,99.0,Christchurch City.,Not Applicable,July 2014,1,1
4,1 Victimisation Time and Place,Theft and Related Offences,July 2014,July 2014,Theft From Retail Premises,Theft (Except Motor Vehicles),Beerescourt.,.,Other Location,900100,Wednesday,99.0,Hamilton City.,Not Applicable,July 2014,1,1


In [63]:
#unique locations
police_data['Territorial Authority'].unique().tolist() 
 
 
 

['Tauranga City.',
 'Ashburton District.',
 'Auckland.',
 'Christchurch City.',
 'Hamilton City.',
 'New Plymouth District.',
 'Palmerston North City.',
 'Upper Hutt City.',
 'Wanganui District.',
 'Horowhenua District.',
 'Lower Hutt City.',
 'Whangarei District.',
 'Porirua City.',
 'Kapiti Coast District.',
 'Whakatane District.',
 'Queenstown-Lakes District.',
 'Gisborne District.',
 'Matamata-Piako District.',
 'Waitaki District.',
 'Dunedin City.',
 'Timaru District.',
 'Napier City.',
 'Tasman District.',
 'Far North District.',
 'Hastings District.',
 'Thames-Coromandel District.',
 'Nelson City.',
 'Stratford District.',
 'Wellington City.',
 'Rotorua District.',
 'Waitomo District.',
 'South Waikato District.',
 'Western Bay of Plenty District.',
 'Ruapehu District.',
 'Masterton District.',
 'Taupo District.',
 'Waikato District.',
 'Mackenzie District.',
 'Invercargill City.',
 'Hauraki District.',
 'Hurunui District.',
 'Kaipara District.',
 'Manawatu District.',
 'Central

In [64]:
#Validating the data
print("Total months since the data is available:",(6+3+12*5) )
print("Unique month-year in data:",len(police_data['Year Month'].unique()))
print("Total number of unique regions in data:", len(police_data['Territorial Authority'].unique()) )

Total months since the data is available: 69
Unique month-year in data: 69
Total number of unique regions in data: 66


### Determining the top 20 districts with highest victimizations (crime)

In [76]:
#Determining the total victimizations grouped by year-month and location (districts)
prepared_police_data = police_data.groupby(['Year Month','Territorial Authority'],as_index=False)

prepared_police_data = prepared_police_data['Victimisations'].sum()
 

locations = police_data['Territorial Authority'].unique()
crime_count = []
for r in locations:
    crime_count.append(prepared_police_data[prepared_police_data['Territorial Authority'].isin([r])]['Victimisations'].sum())

region_crime_count =  pd.DataFrame(list(zip(locations,crime_count)), columns = ['Location','crime_count']).sort_values('crime_count',ascending=False)

location_top20 = region_crime_count['Location'].head(n=20)


location_top20

2                  Auckland.
3         Christchurch City.
4             Hamilton City.
28          Wellington City.
0             Tauranga City.
6     Palmerston North City.
29         Rotorua District.
10          Lower Hutt City.
24        Hastings District.
11       Whangarei District.
21              Napier City.
23       Far North District.
19             Dunedin City.
36         Waikato District.
16        Gisborne District.
5     New Plymouth District.
12             Porirua City.
35           Taupo District.
8         Wanganui District.
26              Nelson City.
Name: Location, dtype: object

In [70]:
region_crime_count.head()

,Location,crime_count
2,Auckland.,452885
3,Christchurch City.,124713
4,Hamilton City.,62543
28,Wellington City.,49231
0,Tauranga City.,36070


### Extracting police data for the 20 locations identified

In [71]:

prepared_police_data_20=prepared_police_data[prepared_police_data['Territorial Authority'].isin(location_top20)]

### Cleaning the Location names to align with those in the vector file.

In [77]:
cleaned_names = []
for r in prepared_police_data['Territorial Authority']:
    r=r.replace('District','')
    r=r.replace('City','')
    r=r.replace('\"','')
    r=r.replace('.','')
    r=r.strip()
     
    
    cleaned_names.append(r)
    
 
prepared_police_data['Territorial Authority'] = cleaned_names
print(prepared_police_data['Territorial Authority'])

0                   Ashburton
1                    Auckland
2                      Buller
3                   Carterton
4         Central Hawke's Bay
                ...          
4549               Wellington
4550    Western Bay of Plenty
4551                 Westland
4552                Whakatane
4553                Whangarei
Name: Territorial Authority, Length: 4554, dtype: object


## Tweet Extraction and Preparation


The tweets have been extracted using TweetScraper (https://github.com/jonbakerfish/TweetScraper). All the tweets from January 2018 till March 2020 sento to @nzpolice (https://twitter.com/nzpolice) Twitter handle have been scraped from Twitter website. A typical query for extracting tweets from Auckland: scrapy crawl TweetScraper -a "query=@nzpolice near:Auckland NZ since:2014-07-01 until:2020-03-31"

In [50]:
import json
import os
import pandas as pd
from tqdm import tqdm_notebook

path = '../data/tweets-all/' 
tweets = []
dates = []
 
for i,filename in enumerate(tqdm_notebook(os.listdir(path ))):
           with open(os.path.join(path , filename), 'r', encoding="utf8") as f:
            content = json.loads(f.read())
            dates.append(content["datetime"])
            tweets.append(content["text"])

            

            
            

#tweet_data_csv_f=open(path+"alltweets.csv", "a+")
    


C:\Users\deepa\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


### Extracting sentiment from the tweets

In [52]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm_notebook
analyser = SentimentIntensityAnalyzer()


sentiment = []

for i,tweet in enumerate(tqdm_notebook(tweets)):
    string = tweet
    string = re.sub(r'https?:\S+', "", string) #Removes Hyperlink
    string = re.sub('@[^\s]+','',string) #Removes Usernames
    string = re.sub('#[^\s]+','',string) #Removes Hashtags
    
    score = analyser.polarity_scores(string)
    score.pop('compound')
    sentiment.append(max(score, key=score.get))

 

C:\Users\deepa\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


### Converting DateTime to YearMonth and YearMonthDay

In [53]:
import datetime
yearmonth = []
yearmonthday = []
for dt in dates:
    date_time_obj = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    yearmonth.append(  str(date_time_obj.year) + '-' + str(date_time_obj.month))
    yearmonthday.append(  str(date_time_obj.year) + '-' + str(date_time_obj.month) + '-' + str(date_time_obj.day) )

### Creating Dataframe for determining aggregates

In [54]:
tweet_sentiment_df =  pd.DataFrame(list(zip(yearmonth,yearmonthday,sentiment)), columns = ['Year Month','Year Month Day','Sentiment'])

 
prepared_tweet_data_year_month_day = tweet_sentiment_df.groupby( 'Year Month Day' )['Sentiment'].value_counts().unstack().fillna(0)
prepared_tweet_data_year_month = tweet_sentiment_df.groupby( 'Year Month' )['Sentiment'].value_counts().unstack().fillna(0)

#display(prepared_tweet_data_year_month)
print(prepared_tweet_data_year_month_day)
 
#tweet_sentiment_df['Year Month'].unique()

Sentiment       neg   neu  pos
Year Month Day                
2014-10-1       0.0  29.0  0.0
2014-10-10      0.0  19.0  0.0
2014-10-11      0.0  26.0  1.0
2014-10-12      0.0   9.0  0.0
2014-10-13      0.0  19.0  0.0
...             ...   ...  ...
2020-5-5        1.0  57.0  9.0
2020-5-6        1.0  62.0  3.0
2020-5-7        2.0  40.0  3.0
2020-5-8        2.0  51.0  6.0
2020-5-9        1.0  30.0  3.0

[2148 rows x 3 columns]


In [56]:
tweet_sentiment_df[tweet_sentiment_df['Sentiment'].isin(['neg'])]['Sentiment'].count()


2216

## Saving the files for use in Shiny/Leaflet

In [74]:
 

prepared_tweet_data_year_month_day.to_csv('../data/tweet_sentiment_ymd.csv', encoding='utf-8')
prepared_tweet_data_year_month.to_csv('../data/tweet_sentiment_ym.csv', encoding='utf-8')
prepared_police_data.to_csv('../data/police_filtered_data.csv', encoding='utf-8')



NameError: name 'prepared_tweet_data_year_month_day' is not defined

In [73]:
## writing grouped data for all cities
## Initially only 20 cities were considered for analysis but later all the cities were considered
region_crime_count.to_csv('../data/police_grouped_data.csv', encoding='utf-8')

In [78]:
prepared_police_data.to_csv('../data/police_filtered_data.csv', encoding='utf-8')
